The goal in this file is comparing the solution of regular Bounded-Makespan-Min-Cost with Fixed_x_Bounded_Makespan_Min_Cost and generate the figure automatically.

In [1]:
# importing packages
import numpy as np
import pandas as pd
from gurobipy import *

from scipy.stats import skewnorm

# from bokeh.io import output_notebook, output_file, curdoc, push_notebook, show
# from bokeh.plotting import figure, show
# from bokeh.models import HoverTool, Slider, ColumnDataSource, CustomJS
# from bokeh.layouts import row, column, gridplot, widgetbox
# from bokeh.models.widgets import Panel, Tabs, Button

# from bokeh.models import Arrow, OpenHead, NormalHead, VeeHead

import numpy as np
import pandas as pd

from gurobipy import *

# output_notebook()

In [2]:
ScnCount = [20, 50, 100, 200, 1000]
subScnCount = 20
maxScn = 1000

Number = range(11, 49)
SubNumber = [1]
directory2 = "./scenarioGameLeftSkew600/10_scenarioGame_" 

In [3]:
# reading data
def readData(readNumber, readSubNumber):  
        jobs = 30
        testNumber = readNumber # 1 to 48
        subTestNumber = readSubNumber # 1 to 10
        fileName = 'J' + str(jobs) + str(testNumber) + '_' + str(subTestNumber) + '.RCP'
        file = '../dataRCPSP/A' + str(jobs) + '/j' + str(jobs) + 'rcp/' + fileName
        my_cols = ["0", "1", "2", "3", "4", "5", "6", "7", "8"]
        df = pd.read_table(file, sep='\t',delim_whitespace=True, engine='python', names=my_cols)
        duration = df.iloc[2:, 0].values
        resourcesUsage = df.iloc[2:, 1:5].values
        successorsCount = df.iloc[2:, 5].values
        successors = df.iloc[2:, 6:9].values
        resourcesCount = int(df.iloc[0, 1])
        jobsCount = int(df.iloc[0,0])
        availableResources = df.iloc[1,0:resourcesCount].values
        return duration, resourcesUsage, successors, resourcesCount, jobsCount, availableResources

In [4]:
# model sets and parameters for FCT model
def FCTSetParam(jobsCount, resourcesCount, duration, resourcesUsage, availableResources, scenarioCount, subScenarioCount):  
        activities = np.array(range(jobsCount))
        resources = np.array(range(resourcesCount))
        scenarios = np.array(range(scenarioCount))
        subScenarios = np.array(range(subScenarioCount))
        T = duration.sum()
        d = dict(zip(activities,resourcesUsage))
        M = 150
        N = availableResources.max()
        return activities, resources, d, T, M , N, scenarios, subScenarios

In [5]:
# # scenario generating Uniform
# def scenarioGen(scenarios, jobsCount, duration, activities):
#     np.random.seed(0)
#     durationScenarios = np.zeros(shape=(jobsCount, scenarios[-1] + 1))
#     aMin = 1 - (12**0.5)/10 #20%
#     aMax = 1 + (12**0.5)/10
#     for s in scenarios:
#         for a in activities:
#             cr = np.random.rand()
#             durationScenarios[a,s] = duration[a] * (aMin + cr * (aMax-aMin))
#     p = dict(zip(activities,durationScenarios))
#     return p



# # scenario generating Normal
# def scenarioGen(scenarios, jobsCount, duration, activities):
#     np.random.seed(0)
#     durationScenarios = np.zeros(shape=(jobsCount, scenarios[-1] + 1))
#     for s in scenarios:
#         for a in activities:
#             if duration[a] != 0:
#                 cr = np.random.normal(duration[a], duration[a]*0.2)
#             else: cr = 0
#             if cr >= 0:
#                 durationScenarios[a,s] = cr
#             else: 
#                 durationScenarios[a,s] = duration[a]
#     p = dict(zip(activities,durationScenarios))
#     return p



# scenario generating Left Skew 600
def scenarioGen(scenarios, jobsCount, duration, activities):
    np.random.seed(0)
    durationScenarios = np.zeros(shape=(jobsCount, scenarios[-1] + 1))
    
    for a in activities[1:-1]:
        rndsk = skewnorm.rvs(600, size=1000)
        rndsk = rndsk * duration[a] * 0.1 /rndsk.std()
        rndsk = rndsk - rndsk.mean() + duration[a]
        for s in scenarios:
            durationScenarios[a,s] = rndsk[s]
    
    p = dict(zip(activities,durationScenarios))
    return p



# # scenario generating Left Skew 8
# def scenarioGen(scenarios, jobsCount, duration, activities):
#     np.random.seed(0)
#     durationScenarios = np.zeros(shape=(jobsCount, scenarios[-1] + 1))
    
#     for a in activities[1:-1]:
#         rndsk = skewnorm.rvs(8, size=1000)
#         rndsk = rndsk * duration[a] *0.2 /rndsk.std()
#         rndsk = rndsk - rndsk.mean() + duration[a]
#         for s in scenarios:
#             durationScenarios[a,s] = rndsk[s]
    
#     p = dict(zip(activities,durationScenarios))
#     return p

In [6]:
# define variables of FCT model
def addFCTVars(activities, resources, T, availableResources):  
        x = FCT.addVars(activities, activities, lb=0.0, ub=1.0, vtype='B', 
                       name="X") # one if activity i complete before activity j starts
        s = FCT.addVars(activities, scenarios, lb=0.0, ub=T, vtype='C', 
                       name="S") # start time of activity i in scenario s
        o = FCT.addVars(activities, lb=0.0, ub=T, vtype='C', 
                       name="O") # due date of procurements for activity i
        e = FCT.addVars(activities, scenarios, lb=0.0, ub=T, vtype='C', 
                       name="E") # earliness of materials for activity i in scenario s
        f = FCT.addVars(activities, activities, resources, lb=0.0, ub=availableResources.max(), vtype='C', 
                       name="F") # amount of resource r that after completion of activity i will pass to activity j
        
        return x, s, f, o, e

In [7]:
# define variables of FCT model
def addFCTVarsMM(activities, resources, T, availableResources):  
        x = FCT.addVars(activities, activities, lb=0.0, ub=1.0, vtype='B', 
                       name="X") # one if activity i complete before activity j starts
        s = FCT.addVars(activities, lb=0.0, ub=T, vtype='C', 
                       name="S") # start time of activity i in scenario s
        f = FCT.addVars(activities, activities, resources, lb=0.0, ub=availableResources.max(), vtype='C', 
                       name="F") # amount of resource r that after completion of activity i will pass to activity j
        
        return x, s, f

In [8]:
# define variables of FCT model
def addFCTVarsXX(activities, resources, T, availableResources):  
        s = FCT.addVars(activities, scenarios, lb=0.0, ub=T, vtype='C', 
                       name="S") # start time of activity i in scenario s
        o = FCT.addVars(activities, lb=0.0, ub=T, vtype='C', 
                       name="O") # due date of procurements for activity i
        e = FCT.addVars(activities, scenarios, lb=0.0, ub=T, vtype='C', 
                       name="E") # earliness of materials for activity i in scenario s
        f = FCT.addVars(activities, activities, resources, lb=0.0, ub=availableResources.max(), vtype='C', 
                       name="F") # amount of resource r that after completion of activity i will pass to activity j
        
        return s, f, o, e

In [9]:
# constraint 2.5 of FCT model
def addFCTConst1(activities, successors, x):    
        FCT.addConstrs(
            (x[activityI, activityJ] 
             ==
             1
             for activityI in activities if activityI != activities[-1]
             for activityJ in activities[successors[activityI][successors[activityI] > 0].astype(np.int) - 1]), 
            name = "NetworkRelations")

In [10]:
# constraint 2.6a of FCT model
def addFCTConst2a(activities, x, s, p, M, scenarios):  
        FCT.addConstrs(
            (s[activityJ, scenario] -  s[activityI, scenario] 
             >=
             p[activityI][scenario] - M * (1 - x[activityI, activityJ])
             for scenario in scenarios
             for activityI in activities if activityI != activities[-1] 
             for activityJ in activities if activityJ != activities[0]), 
            name = "NetworkStartTimeRelations")

In [11]:
# constraint 2.6a of FCT model
def addFCTConst2aMM(activities, x, s, ppp, M):  
        FCT.addConstrs(
            (s[activityJ] -  s[activityI] 
             >=
             ppp[activityI] - M * (1 - x[activityI, activityJ])
             for activityI in activities if activityI != activities[-1] 
             for activityJ in activities if activityJ != activities[0]), 
            name = "NetworkStartTimeRelations")

In [12]:
# constraint 2.6b of FCT model
def addFCTConst2b(activities, s, o, scenarios):  
        FCT.addConstrs(
            (s[activity, scenario] 
             >=
             o[activity]
             for scenario in scenarios
             for activity in activities), 
            name = "NetworkStartTimeAndOrderRelations")

In [13]:
# constraint 2.7 of FCT model
def addFCTConst3(activities, x, f, resources, N):  
        FCT.addConstrs(
            (f[activityI, activityJ, resource] -  N * x[activityI, activityJ] 
             <=
             0
             for activityI in activities if activityI != activities[-1] 
             for activityJ in activities if activityJ != activities[0]
             for resource in resources), 
            name = "NetworkFlowRelations")

In [14]:
# constraint 2.8 of FCT model
def addFCTConst4(activities, f, resources, d):  
        FCT.addConstrs(
            (quicksum(f[activityI, activityJ, resource] 
                      for activityJ in activities 
                      if activityJ != activities[0]) 
             ==
             d[activityI][resource]
             for activityI in activities if activityI != activities[0] 
             for resource in resources), 
            name = "OutgoingFlows")

In [15]:
# constraint 2.9 of FCT model
def addFCTConst5(activities, f, resources, d):  
        FCT.addConstrs(
            (quicksum(f[activityI, activityJ, resource] 
                      for activityI in activities 
                      if activityI != activities[-1]) 
             ==
             d[activityJ][resource]
             for activityJ in activities if activityJ != activities[-1] 
             for resource in resources), 
            name = "IngoingFlows")

In [16]:
# constraint 2.10a of FCT model
def addFCTConst6(activities, f, resources, availableResources):  
        FCT.addConstrs(
            (quicksum(f[activities[0], activityJ, resource] 
                      for activityJ in activities 
                      if activityJ != activities[0]) 
             ==
             availableResources[resource]
             for resource in resources), 
            name = "FirstFlow")

In [17]:
# constraint 2.10b of FCT model
def addFCTConst7(activities, f, resources, availableResources):  
        FCT.addConstrs(
            (quicksum(f[activityI, activities[-1], resource] 
                      for activityI in activities 
                      if activityI != activities[-1]) 
             ==
             availableResources[resource]
             for resource in resources), 
            name = "LastFlow")

In [18]:
# constraint for bounding makespan
def addFCTConst9(activities, scenarios, scenarioCount, s, makespan):  
        FCT.addConstrs(
            (quicksum(s[activities[-1], scenario] for scenario in scenarios)
             <=
             makespan * scenarioCount
             for alaki in range(0,1)
             ), 
            name = "Makespandddd")

In [19]:
# Objective of FCT model -- min cost
def addFCTObj(activities, s, scenarios, scenarioCount, o): 
        obj = (quicksum(s[activity, scenario] - o[activity] 
                              for activity in activities
                              for scenario in scenarios) / scenarioCount)
        FCT.setObjective(obj, GRB.MINIMIZE)

In [20]:
# Objective of FCT model -- min makespan det
def addFCTObjMM(activities, s):
  
        obj = (s[activities[-1]])
        FCT.setObjective(obj, GRB.MINIMIZE)

In [ ]:
# Objective of FCT model -- min expected makespan
def addFCTObjdW(activities, s, scenarios, scenarioCount):
  
        obj = (quicksum(s[activities[-1], scenario] for scenario in scenarios) / scenarioCount)
        FCT.setObjective(obj, GRB.MINIMIZE)

In [ ]:
for i in Number:
    for j in SubNumber: 
        
        
        try:


            #MM##########################################################################################################
            ################################# MIN makespan for DETERMINISTIC
            print("iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii_" + str(i) + "_" + str(j))
            print("    MIN makespan for DETERMINISTIC    step=1  :   " + str(i) + "_" + str(j))
            print("iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii")
            ############################################################################################################1

            makespanDet = 0
            makespanExp = 0

            duration, resourcesUsage, successors, resourcesCount, jobsCount, availableResources = readData(i, j)
            activities, resources, d, T, M , N, scenarios, subScenarios = \
                FCTSetParam(jobsCount, resourcesCount, duration, resourcesUsage, availableResources, maxScn, subScnCount)
            pAll = scenarioGen(scenarios, jobsCount, duration, activities)
            pMM = list(range(jobsCount))
            for act in range(jobsCount):
                pMM[act] = sum(pAll[act]) / maxScn
            FCT = Model('FCT: MIN makespan for DETERMINISTIC')
            FCT.setParam('TimeLimit', 60)
            x, s, f = addFCTVarsMM(activities, resources, T, availableResources)
            addFCTConst1(activities, successors, x)
            addFCTConst2aMM(activities, x, s, pMM, M)
            addFCTConst3(activities, x, f, resources, N)
            addFCTConst4(activities, f, resources, d)
            addFCTConst5(activities, f, resources, d)
            addFCTConst6(activities, f, resources, availableResources)
            addFCTConst7(activities, f, resources, availableResources)
            FCT.update()
            addFCTObjMM(activities, s)
            FCT.optimize()        

    #.............................................. Save MM-X

            directory = directory2 + str(i) + "_" + str(j) + "/"
            !mkdir $directory
            if FCT.Status == 2:

                # Open a file X
                xoName = directory+ "MM-x.txt"
                xo = open(xoName, "a")
                for v in range(jobsCount):
                    if v != 0 : xo.write("\n")
                    for k in range(jobsCount):
                        xo.write(str(int(x[v, k].X)) + "\t")
                # Close opend file X
                xo.close()

    #.............................................. Update eff.txt

            preName =  "|A" + str(jobsCount-2) + "|" + str(i) + "_" + str(j) + "|MM|\t\t"
            if FCT.Status == 2:
                # Open a file
                do = open(directory + "eff.txt", "a")
                do.write(preName)
                do.write(str(round(FCT.ObjVal,4)) + "\t\t" + str(round(FCT.Runtime,1)) + " sec\n")
                # Close opend file
                do.close()
            else:
                # Open a file
                do = open(directory + "eff.txt", "a")
                do.write(preName)
                do.write("Unable to find the optimum solution in " + str(round(FCT.Runtime,1)) + " sec\n")
                # Close opend file
                do.close()       


            #dW############################################################################################################
            ##################### MIN E(makespan) for SCENARIO base | XdW bede
            print("iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii_" + str(i) + "_" + str(j))
            print("    MIN E(makespan) for SCENARIO base | XdW bede   step=2  :  " + str(i) + "_" + str(j))
            print("iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii")
            ##############################################################################################################2

            subP = dict(zip(activities,np.array(list(pAll.values()))[:,:subScnCount]))
            FCT = Model('FCT: MIN makespan for SCENARIO base')
            FCT.setParam('TimeLimit', 600)
            x, s, f, o, e = addFCTVars(activities, resources, T, availableResources)
            addFCTConst1(activities, successors, x)
            addFCTConst2a(activities, x, s, subP, M, subScenarios)
            addFCTConst2b(activities, s, o, subScenarios)
            addFCTConst3(activities, x, f, resources, N)
            addFCTConst4(activities, f, resources, d)
            addFCTConst5(activities, f, resources, d)
            addFCTConst6(activities, f, resources, availableResources)
            addFCTConst7(activities, f, resources, availableResources)
            FCT.update()
            addFCTObjdW(activities, s, subScenarios, subScnCount)
            FCT.optimize()        

    #.............................................. Save dW-X

            if FCT.Status == 2:
                # Open a file X
                xoName = directory + "dW-x.txt"
                xo = open(xoName, "a")
                for v in range(jobsCount):
                    if v != 0 : xo.write("\n")
                    for k in range(jobsCount):
                        xo.write(str(int(x[v, k].X)) + "\t")
                # Close opend file X
                xo.close()

    #.............................................. Update eff.txt

            # Calculations
            if FCT.MIPGap <= 10000:
                aaa = 0 # Expected Makespan
                for vvv in range(subScnCount):
                    aaa += s[jobsCount - 1, vvv].X
            preName =  "|A" + str(jobsCount-2) + "|" + str(i) + "_" + str(j) + "|dW|\t\t"
            if FCT.Status == 2:
                # Open a file
                do = open(directory + "eff.txt", "a")
                do.write(preName)
                do.write(str(round(aaa / subScnCount,4)) + "\t\t" + str(round(FCT.Runtime,1)) + " sec\n")
                # Close opend file
                do.close()
            else: 
                # Open a file
                do = open(directory + "eff.txt", "a")
                do.write(preName)
                do.write("Unable to find the optimum solution in " + str(round(FCT.Runtime,1)) + " sec\n")
                # Close opend file
                do.close()                 





            for scnCount in ScnCount:


                #SdW###############################################################################################################
                ####################################   for fixed X: MIN E(makespan) | use x of dW shema bede
                print("iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii_" + str(i) + "_" + str(j))
                print("  for fixed X: MIN E(makespan) | use x of dW shema bede   step=3  :  " + str(i) + "_" + str(j) + " || " + str(scnCount))
                print("iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii")
                #################################################################################################################3

                fileName =  "dW-x.txt"
                file = directory + fileName
                x = np.loadtxt(file)
                x = x.astype(int)


                p = dict(zip(activities,np.array(list(pAll.values()))[:,:scnCount]))


                FCT = Model('for fixed X: MIN scenario used makespan | use x of dW')
                FCT.setParam('TimeLimit', 10*60)
                s, f, o, e = addFCTVarsXX(activities, resources, T, availableResources)
                addFCTConst2a(activities, x, s, p, M, scenarios[:scnCount])
                addFCTConst2b(activities, s, o, scenarios[:scnCount])
                addFCTConst3(activities, x, f, resources, N)
                addFCTConst4(activities, f, resources, d)
                addFCTConst5(activities, f, resources, d)
                addFCTConst6(activities, f, resources, availableResources)
                addFCTConst7(activities, f, resources, availableResources)
                FCT.update()
                addFCTObjdW(activities, s, scenarios[:scnCount], scnCount)
                FCT.optimize()

        #.............................................. Update eff.txt

                # Calculations
                preName = "|A" + str(jobsCount-2) + "|" + str(i) + "_" + str(j) + "|SdW|" + str(scnCount) + "\t\t"
                if FCT.Status == 2:
                    # Open a file
                    makespanExp = FCT.ObjVal
                    do = open(directory + "eff.txt", "a")
                    do.write(preName)
                    do.write(str(round(FCT.ObjVal,4)) + "\t\t" + str(round(FCT.Runtime,1)) + " sec\n")
                    # Close opend file
                    do.close()
                else:  
                    # Open a file
                    do = open(directory + "eff.txt", "a")
                    do.write(preName)
                    do.write("Unable to find the optimum solution in " + str(round(FCT.Runtime,1)) + " sec\n")
                    # Close opend file
                    do.close() 



                #SMM###############################################################################################################
                ####################################   for fixed X: MIN E(makespan) | use x of MM shema bede
                print("iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii_" + str(i) + "_" + str(j))
                print("  for fixed X: MIN E(makespan) | use x of MM shema bede   step=4  :  " + str(i) + "_" + str(j) + "  || " + str(scnCount))
                print("iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii")
                #################################################################################################################4

                fileName =   "MM-x.txt"
                file = directory + fileName
                x = np.loadtxt(file)
                x = x.astype(int)

                FCT = Model('for fixed X: MIN scenario used makespan | use x of MM')
                FCT.setParam('TimeLimit', 10*60)
                s, f, o, e = addFCTVarsXX(activities, resources, T, availableResources)
                addFCTConst2a(activities, x, s, p, M, scenarios[:scnCount])
                addFCTConst2b(activities, s, o, scenarios[:scnCount])
                addFCTConst3(activities, x, f, resources, N)
                addFCTConst4(activities, f, resources, d)
                addFCTConst5(activities, f, resources, d)
                addFCTConst6(activities, f, resources, availableResources)
                addFCTConst7(activities, f, resources, availableResources)
                FCT.update()
                addFCTObjdW(activities, s, scenarios[:scnCount], scnCount)
                FCT.optimize()

        #.............................................. Update eff.txt

                # Calculations
                preName =  "|A" + str(jobsCount-2) + "|" + str(i) + "_" + str(j) + "|SMM|" + str(scnCount) + "\t\t"
                if FCT.Status == 2:
                    # Open a file
                    makespanDet = FCT.ObjVal
                    do = open(directory + "eff.txt", "a")
                    do.write(preName)
                    do.write(str(round(FCT.ObjVal,4)) + "\t\t" + str(round(FCT.Runtime,1)) + " sec\n")
                    # Close opend file
                    do.close()
                else:  
                    # Open a file
                    do = open(directory + "eff.txt", "a")
                    do.write(preName)
                    do.write("Unable to find the optimum solution in " + str(round(FCT.Runtime,1)) + " sec\n")
                    # Close opend file
                    do.close() 



        except Exception as err:
            print("Erro {}".format(err))

iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii_11_1
    MIN makespan for DETERMINISTIC    step=1  :   11_1
iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii
Changed value of parameter TimeLimit to 60.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Optimize a model with 5109 rows, 5152 columns and 18495 nonzeros
Variable types: 4128 continuous, 1024 integer (1024 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 2e+02]
  RHS range        [1e+00, 2e+02]
Presolve removed 2563 rows and 2651 columns
Presolve time: 0.04s
Presolved: 2546 rows, 2501 columns, 8994 nonzeros
Variable types: 1841 continuous, 660 integer (660 binary)

Root relaxation: objective 5.200000e+01, 1329 iterations, 0.06 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   52.00000    0   21          -   52.00000    

 26131  9735   63.11577   89   18   64.51974   52.51034  18.6%  46.6   60s
 26140  9741   55.42835   96   36   64.51974   52.51738  18.6%  46.6   65s
H26140  9253                      64.3064456   52.51738  18.3%  46.6   66s
H26142  8790                      63.1087762   52.51756  16.8%  46.6   68s
 26147  8794   63.10878  103   13   63.10878   52.53831  16.7%  46.6   70s
H26150  8356                      61.8832997   52.56107  15.1%  46.6   72s
 26157  8360   61.88330  106   18   61.88330   52.58259  15.0%  46.6   75s
 26166  8366   56.79299  100   30   61.88330   52.58411  15.0%  46.6   80s
 26171  8370   61.88330   89   15   61.88330   52.58758  15.0%  46.6   85s
 26180  8377   55.42835   96    9   61.88330   52.58758  15.0%  52.1   91s
 26185  8381   57.33116   56   15   61.88330   52.58758  15.0%  52.1   95s
 26190  8384   55.73235  162   34   61.88330   52.58758  15.0%  52.1  100s
 26195  8387   57.98639   88   18   61.88330   52.58827  15.0%  52.0  105s
 26199  8390   57.75837  

H    0     0                      47.0000000   47.00000  0.00%     -    2s
     0     0   47.00000    0    7   47.00000   47.00000  0.00%     -    2s

Cutting planes:
  Gomory: 8
  Cover: 5
  Implied bound: 13
  Clique: 2
  MIR: 53
  Flow cover: 40

Explored 1 nodes (16522 simplex iterations) in 2.93 seconds
Thread count was 24 (of 24 available processors)

Solution count 10: 47 48 50 ... 66

Optimal solution found (tolerance 1.00e-04)
Best objective 4.700000000000e+01, best bound 4.700000000000e+01, gap 0.0000%
iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii_12_1
    MIN E(makespan) for SCENARIO base | XdW bede   step=2  :  12_1
iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii
Changed value of parameter TimeLimit to 600.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Optimize a model with 24008 rows, 69152 columns and 73412 nonzeros
Variable types: 68128 continuous, 1024 integer (1024 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [5e-0



Solved with dual simplex
Solved in 1531 iterations and 0.48 seconds
Optimal objective  4.904236358e+01
iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii_12_1
  for fixed X: MIN E(makespan) | use x of dW shema bede   step=3  :  12_1 || 100
iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii
Changed value of parameter TimeLimit to 600.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Optimize a model with 103400 rows, 68128 columns and 204380 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-02, 1e-02]
  Bounds range     [4e+01, 2e+02]
  RHS range        [9e-01, 1e+02]

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Presolve removed 59201 rows and 64306 columns
Presolve time: 0.59s
Presolved: 44199 rows, 36570 columns, 122302 nonzeros

Ordering time: 0.22s

Barrier statistics:
 AA' NZ     : 1.220e+06
 Factor NZ  : 7.009e+06 (roughly 90 MBytes of memory)
 Factor Ops : 1.388e+09 (less than 1 seco


Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Presolve removed 36707 rows and 624188 columns (presolve time = 5s) ...
Presolve removed 36707 rows and 624189 columns
Presolve time: 7.82s
Presolved: 31421 rows, 441039 columns, 882095 nonzeros

Ordering time: 0.03s

Barrier statistics:
 Free vars  : 9
 AA' NZ     : 3.332e+05
 Factor NZ  : 5.162e+05 (roughly 200 MBytes of memory)
 Factor Ops : 1.169e+07 (less than 1 second per iteration)
 Threads    : 22

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -3.05566979e+06 -0.00000000e+00  2.90e+00 1.00e+02  2.08e+01     9s
   1  -1.44545186e+06  1.84364620e+02  2.40e-01 3.61e+01  5.03e+00     9s
   2  -2.50693312e+05  1.74434986e+02  6.73e-03 1.11e+00  5.80e-01     9s
   3  -3.50113282e+04  1.74133291e+02  8.71e-04 1.87e-01  7.96e-02     9s
   4  -3.15155733e+03  1.73705735e+02  6.90e-05 1.72e-02  7.49e-03    1

...

Presolve time: 0.15s
Presolved: 5784 rows, 2675 columns, 14532 nonzeros

Ordering time: 0.00s

Barrier performed 0 iterations in 0.17 seconds
Barrier solve interrupted - model solved by another algorithm


Solved with dual simplex
Solved in 768 iterations and 0.20 seconds
Optimal objective  5.801248246e+01
iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii_47_1
  for fixed X: MIN E(makespan) | use x of MM shema bede   step=4  :  47_1  || 20
iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii
Changed value of parameter TimeLimit to 600.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Optimize a model with 23960 rows, 68128 columns and 50300 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-02, 5e-02]
  Bounds range     [2e+01, 2e+02]
  RHS range        [9e-01, 1e+02]

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Presolve removed 17891 rows and 66167 columns
Presolve time: 0.17s
Presolved: 6069

   0   1.30425813e+02 -1.04602424e+04  4.06e+02 6.94e-18  1.45e+00     1s
   1   2.15389780e+02 -7.64752645e+03  1.10e+02 1.66e-02  3.44e-01     1s
   2   1.53355276e+02 -2.65928421e+03  2.05e+00 2.13e-15  3.67e-02     1s

Barrier performed 2 iterations in 1.27 seconds
Barrier solve interrupted - model solved by another algorithm


Solved with dual simplex
Solved in 5672 iterations and 1.55 seconds
Optimal objective  5.961776216e+01
iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii_47_1
  for fixed X: MIN E(makespan) | use x of dW shema bede   step=3  :  47_1 || 1000
iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii
Changed value of parameter TimeLimit to 600.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Optimize a model with 997100 rows, 68128 columns and 1937780 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-03, 1e-03]
  Bounds range     [2e+01, 2e+02]
  RHS range        [9e-01, 1e+02]

Concurrent LP optimizer: primal simplex, dual

H    0     0                      83.0000000   63.00000  24.1%     -    0s
     0     0   63.00000    0   15   83.00000   63.00000  24.1%     -    0s
H    0     0                      81.0000000   63.00000  22.2%     -    0s
     0     0   63.00000    0   15   81.00000   63.00000  22.2%     -    0s
H    0     0                      63.0000000   63.00000  0.00%     -    0s
     0     0   63.00000    0    3   63.00000   63.00000  0.00%     -    0s

Cutting planes:
  Gomory: 12
  Cover: 3
  Implied bound: 11
  MIR: 36
  Flow cover: 40

Explored 1 nodes (4700 simplex iterations) in 0.49 seconds
Thread count was 24 (of 24 available processors)

Solution count 5: 63 81 83 ... 107

Optimal solution found (tolerance 1.00e-04)
Best objective 6.300000000000e+01, best bound 6.300000000000e+01, gap 0.0000%
iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii_48_1
    MIN E(makespan) for SCENARIO base | XdW bede   step=2  :  48_1
iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii
Changed value of paramet

   1   6.38340921e+01  5.91719072e+01  7.95e+00 4.16e-17  1.57e-02     0s
   2   6.38340921e+01  6.26691593e+01  7.22e-01 2.43e-17  1.95e-03     0s
   3   6.38340921e+01  6.36596369e+01  5.53e-02 1.73e-17  2.16e-04     0s
   4   6.38340921e+01  6.38299211e+01  1.01e-03 1.15e-17  4.68e-06     0s
   5   6.38340921e+01  6.38340919e+01  1.04e-08 8.30e-18  2.79e-10     0s
   6   6.38340921e+01  6.38340921e+01  7.66e-12 2.78e-17  2.87e-16     0s

Barrier solved model in 6 iterations and 0.14 seconds
Optimal objective 6.38340921e+01

Crossover log...

      22 DPushes remaining with DInf 0.0000000e+00                 0s
       0 DPushes remaining with DInf 0.0000000e+00                 0s

     256 PPushes remaining with PInf 0.0000000e+00                 0s
       0 PPushes remaining with PInf 0.0000000e+00                 0s

  Push phase complete: Pinf 0.0000000e+00, Dinf 0.0000000e+00      0s

Iteration    Objective       Primal Inf.    Dual Inf.      Time
     273    6.3834092e+01   0.00

Changed value of parameter TimeLimit to 600.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Optimize a model with 202700 rows, 68128 columns and 396980 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-03, 5e-03]
  Bounds range     [4e+01, 1e+02]
  RHS range        [9e-01, 1e+02]

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Presolve removed 202520 rows and 67671 columns
Presolve time: 0.24s
Presolved: 180 rows, 457 columns, 910 nonzeros

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 4.530e+02
 Factor NZ  : 2.205e+03
 Factor Ops : 3.100e+04 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   6.37350960e+01  5.67975835e+01  7.54e+01 0.00e+00  1.21e-01     0s
   1   6.37350960e+01  5.90729111e+01  7.95e+00 4.16e-17  1.57e-02     0s
   2   6.3735096

In [ ]:
print("FINISHED")